In [20]:
import urllib.request
import json
import pandas as pd
import requests
import shutil
import os
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans



url = (
    "https://pixabay.com/api/?key=42732485-b4c1847a3529ea910e8e5fdeb&q=animaux&image_type=photo&per_page=100"
)
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))
responsedata

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_link = os.path.basename(url)
        filename = os.path.join("./images/", image_link)
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []

for result in responsedata["hits"]:
    array.append(
        (
            result["id"],
            result["tags"],
            result["largeImageURL"],
            result["imageWidth"],
            result["imageHeight"],
            result["imageSize"],
            result["likes"], 
            result["user"],
            result["largeImageURL"].split('.')[-1]
        )
    )


dataframe = pd.DataFrame(array, columns=["id", "tags", "largeImageURL", "imageWidth","imageHeight","imageSize", "likes", "user", "format"])
dataframe = dataframe.astype(
    dtype={"id": "<U200", "tags": "<U200", "largeImageURL": "<U200","imageWidth": "<U200","imageHeight": "<U200","imageSize": "<U200", "likes": "<U200", "user": "<U200", "format": "<U200"}
)

# Color column
all_images=os.listdir("./images") #list of all the images' names
image_color=[] #list to add the images' color into the dataframe
dataframe['color']='None'
colorsOfAllImages=[] 
for i in range(len(all_images)):
    colorsOfOneImage="" #Reinitialization of the value of the colors
    color = [] #most significant colors of the images
    imgfile = Image.open("./images/"+all_images[i])
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    if numarray.ndim == 1:
        imgfile = imgfile.convert('RGB')
        numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    clusters = KMeans(n_clusters=4, n_init=2)
    clusters.fit(numarray)
    npbins = numpy.arange(0, 5)
    histogram = numpy.histogram(clusters.labels_, bins=npbins)
    labels = numpy.unique(clusters.labels_)
    for i in range(4):
        image_color=[math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2])]
        # Categorization
        for j in range(3):
            image_color[j] -= (image_color[j]) % 32
        color.append(image_color)
    colorsOfAllImages.append(color)
dataframe.loc[:,'color']=colorsOfAllImages

# Data type conversion to sort
# Int conversion
for int_column in ["id","imageWidth","imageHeight","imageSize","likes"]:
    dataframe[int_column] = dataframe[int_column].astype(int)
# "tags" conversion into list
new_tags = []
for i in range(len(dataframe["tags"])):
    new_tags.append(dataframe.loc[i, "tags"].replace(" ","").split(','))
dataframe['tags']=new_tags
    
    
    
# imageSize Categorization
dataframe["imageSize"] -= dataframe["imageSize"] % 500000 
for i in range(len(dataframe["imageSize"])):
    if dataframe.loc[i, "imageSize"] > 2000000:
        dataframe.loc[i, "imageSize"] = 2000000
dataframe



# Json conversion, Storage
json_data = dataframe.to_json(orient='records', indent = 4)
file_path = "data.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

# Read test for the json produced
readtest_data = json.load(open("data.json"))
readtest_dataframe = pd.json_normalize(readtest_data)
readtest_dataframe

,id,tags,largeImageURL,imageWidth,imageHeight,imageSize,likes,user,format,color
0,715588,"[fox, nature, animals]",https://pixabay.com/get/gcb371a691b0b3a1db0ad0...,3595,2414,2226561,1513,AlainAudet,jpg,"[[192, 160, 32], [224, 192, 128], [32, 32, 32]..."
1,430441,"[horses, galloping, running]",https://pixabay.com/get/gd55dc79b1c02314105760...,4500,3000,1347483,1522,SorcerySoapHP,jpg,"[[160, 128, 128], [96, 64, 64], [224, 224, 224..."
2,591359,"[tigers, cub, snow]",https://pixabay.com/get/g3763ede63f78aa5c64ba1...,4342,2798,5424078,1746,Sponchia,jpg,"[[160, 0, 0], [160, 160, 96], [224, 224, 192],..."
3,2904536,"[horses, pair, wildhorses]",https://pixabay.com/get/g002ebc05c49c5ae5fe59e...,4000,3000,2640822,1590,susannp4,jpg,"[[0, 0, 0], [192, 96, 0], [128, 32, 0], [224, ..."
4,2934720,"[cat, kitten, pets]",https://pixabay.com/get/gd697e6ed9b3349294abac...,3495,2330,1971228,1549,Kirgiz03,jpg,"[[64, 64, 0], [160, 128, 96], [128, 96, 32], [..."
...,...,...,...,...,...,...,...,...,...,...
95,2222908,"[giraffe, head, ossicones]",https://pixabay.com/get/g73a3cf54b0f0c1915b9de...,4000,2669,3727985,689,Alexas_Fotos,jpg,"[[160, 96, 32], [96, 96, 96], [192, 160, 128],..."
96,2045655,"[eagle, portrait, wild]",https://pixabay.com/get/g182b00c6f4e923e0ba397...,2066,2426,1388406,852,Robert_C,jpg,"[[32, 32, 0], [224, 192, 160], [128, 96, 64], ..."
97,1341881,"[wolves, forest, wintry]",https://pixabay.com/get/ge42e5e4b61c01d764f8ff...,4096,2160,1380594,3164,51581,jpg,"[[96, 128, 96], [32, 64, 0], [160, 192, 128], ..."
98,1869337,"[dolphins, mammals, animals]",https://pixabay.com/get/g2fce9c1c2599a1067b4f8...,3840,2160,886445,388,Pexels,jpg,"[[0, 0, 0], [224, 224, 224], [64, 64, 64], [12..."


On va stocker id, tags, largeImageURL, imageWidth, imageHeight, imageSize, likes, format de l'image en prenant ce qu'il y a après le .; user

In [21]:
import json
import pandas as pd

# Loads json
data = json.load(open("data.json"))
dataframe = pd.json_normalize(data)
#dataframe



# Sort
sort_key = input("Sort by : \n id / imageWidth / imageHeight / imageSize / color / likes ")
dataframe = dataframe.sort_values(by=sort_key)
# Json conversion, Storage
json_data = dataframe.to_json(orient='records', indent = 4)
file_path = "data.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)



# Occurrences
def occurrences(column, isColor = False):
    res = {}
    for value in dataframe[column]:
        if isinstance(value, list): # List case
            for element in value:
                if isColor : # color case
                    element = tuple(element)
                if element in res:
                    res[element] += 1
                else:
                    res[element] = 1
        else: # Single element case
            if value in res:
                res[value] += 1
            else:
                res[value] = 1
    return res
        



print("imageSize",occurrences("imageSize"))
print("color",occurrences("color",True))
print("tags",occurrences("tags"))


imageSize {0: 5, 500000: 15, 1000000: 16, 1500000: 10, 2000000: 54}
color {(64, 224, 224): 1, (0, 128, 128): 1, (128, 32, 32): 1, (0, 32, 32): 3, (160, 160, 192): 1, (64, 64, 32): 13, (160, 96, 64): 2, (224, 192, 160): 5, (128, 64, 64): 2, (224, 224, 224): 15, (64, 32, 32): 11, (160, 160, 128): 10, (0, 0, 0): 20, (96, 64, 64): 6, (160, 128, 96): 10, (224, 160, 0): 1, (0, 96, 128): 1, (96, 160, 192): 1, (96, 128, 160): 2, (160, 192, 224): 5, (32, 64, 96): 2, (128, 96, 64): 12, (192, 128, 96): 2, (224, 224, 192): 5, (96, 64, 32): 7, (96, 96, 96): 11, (32, 32, 32): 18, (160, 160, 160): 14, (128, 128, 128): 7, (64, 64, 64): 10, (192, 192, 192): 15, (160, 160, 96): 2, (192, 160, 128): 10, (192, 192, 128): 2, (128, 128, 96): 9, (128, 160, 224): 1, (192, 224, 224): 2, (160, 128, 128): 5, (192, 192, 160): 12, (96, 96, 64): 7, (128, 128, 64): 2, (96, 96, 32): 5, (128, 160, 32): 1, (32, 32, 0): 21, (160, 192, 160): 2, (64, 96, 32): 3, (128, 160, 160): 1, (96, 128, 128): 3, (128, 96, 32): 4, (32,